In [85]:
import pandas as pd
import os
import json
import psycopg2

In [145]:
conn=psycopg2.connect(host="localhost",
                      user="postgres",
                      password="Kavidhina@5566",
                      port=5432,
                      database="phonepe_pulse")
cursor=conn.cursor()

In [155]:
all_data=migrate_alldata_tosql()

In [144]:
def migrate_alldata_tosql():
    df1=complete_aggregated_datas()
    create_sql_table_aggregated()
    migrate_sql_table_aggregated()
    df2=complete_map_datas()
    create_sql_table_map()
    migrate_sql_table_map()
    df3=complete_top_datas()
    create_sql_table_top()
    migrate_sql_table_top()
    return

In [86]:
def complete_aggregated_datas():
    #aggregated
    # 1.aggregated trasaction
    aggre_trans_path = "D:/Data Science/Git_Clone/Data/pulse/data/aggregated/transaction/country/india/"
    aggre_trans_list = os.listdir(aggre_trans_path) 

    #1.1 year wise aggregated trasaction
    aggre_trans_year_wise = dict (categories=[],year=[],quarter=[],count=[],amount=[])
    for year in range(len(aggre_trans_list)-1):
        json_path = aggre_trans_path+aggre_trans_list[year]+"/"
        json_file_list = os.listdir(json_path)
        quarter = ["Q1","Q2","Q3","Q4"]
        for json_file in range(len(json_file_list)):
            with open(f"{json_path + json_file_list[json_file]}") as df:
                file = json.load(df)
            aggre_trans_year_wise['categories'].append(file['data']['transactionData'][0]['name'])
            aggre_trans_year_wise['year'].append(aggre_trans_list[year])
            aggre_trans_year_wise['quarter'].append(quarter[json_file])
            aggre_trans_year_wise['count'].append(file['data']['transactionData'][0]['paymentInstruments'][0]['count'])
            aggre_trans_year_wise['amount'].append(file['data']['transactionData'][0]['paymentInstruments'][0]['amount'])

    #1.2 state wise aggregated trasaction
    aggre_trans_state_wise = dict (categories=[],year=[],quarter=[],state=[],count=[],amount=[]) 
    state_path = aggre_trans_path+"state"+"/"
    state_list = os.listdir(state_path)
    for state in range(len(state_list)):
        year_path = state_path+state_list[state]+"/"
        years_list = os.listdir(year_path)
        for year in range(len(years_list)):
            json_path = year_path+"/"+years_list[year]
            json_file_list = os.listdir(json_path)
            for json_file in range (len(json_file_list)):
                df = open(f"{json_path+'/'+json_file_list[json_file]}")
                file = json.load(df)            
                quarter = ["Q1","Q2","Q3","Q4"]  
                aggre_trans_state_wise['categories'].append(file['data']['transactionData'][0]['name'])
                aggre_trans_state_wise['year'].append(years_list[year])
                aggre_trans_state_wise['state'].append(state_list[state])
                aggre_trans_state_wise['quarter'].append(quarter[json_file])
                aggre_trans_state_wise['count'].append(file['data']['transactionData'][0]['paymentInstruments'][0]['count'])
                aggre_trans_state_wise['amount'].append(file['data']['transactionData'][0]['paymentInstruments'][0]['amount'])
            
    #2.aggregated user
    aggre_user_path = "D:/Data Science/Git_Clone/Data/pulse/data/aggregated/user/country/india/"
    aggre_user_list = os.listdir(aggre_user_path) 
    #2.1 year wise aggregated user
    aggre_user_year_wise = dict(registered_user=[],app_open=[],year=[],quarter=[],brand=[],count=[],percentage=[])
    for year in range(len(aggre_user_list)-1):
        json_path = aggre_user_path+aggre_user_list[year]+"/"
        json_file_list = os.listdir(json_path)
        for json_file in range(len(json_file_list)):
            df = open(f"{json_path+json_file_list[json_file]}")
            file = json.load(df)
            quarter = ["Q1","Q2","Q3","Q4"]
            usersByDevice_list = file['data']['usersByDevice']
            if usersByDevice_list is not None:
                for userBydevice in range (len(usersByDevice_list)):
                    aggre_user_year_wise['registered_user'].append(file['data']['aggregated']['registeredUsers'])
                    aggre_user_year_wise['app_open'].append(file['data']['aggregated']['appOpens'])
                    aggre_user_year_wise['year'].append(aggre_user_list[year])
                    aggre_user_year_wise['quarter'].append(quarter[json_file])            
                    aggre_user_year_wise['brand'].append(usersByDevice_list[userBydevice]['brand'])
                    aggre_user_year_wise['count'].append(usersByDevice_list[userBydevice]['count'])
                    aggre_user_year_wise['percentage'].append(usersByDevice_list[userBydevice]['percentage'])
            else:
                    aggre_user_year_wise['registered_user'].append(file['data']['aggregated']['registeredUsers'])
                    aggre_user_year_wise['app_open'].append(file['data']['aggregated']['appOpens'])
                    aggre_user_year_wise['year'].append(aggre_user_list[year])
                    aggre_user_year_wise['quarter'].append(quarter[json_file])            
                    aggre_user_year_wise['brand'].append(None)
                    aggre_user_year_wise['count'].append(0)
                    aggre_user_year_wise['percentage'].append(None)
            
    #2.2 state wise aggregated user    
    aggre_user_state_wise = dict(registered_user=[],app_open=[],year=[],quarter=[],state=[],brand=[],count=[],percentage=[])
    state_path=aggre_user_path+"state"+"/"
    state_list=os.listdir(state_path) 
    for state in state_list:
        year_path = state_path+state+"/"
        year_list=os.listdir(year_path)
        for year in range(len(year_list)):
            json_path=year_path+year_list[year]+"/"
            json_list=os.listdir(json_path)
            for json_file in range(len(json_list)):
                df=open(f"{json_path+json_list[json_file]}")
                file=json.load(df)
            quarter = ["Q1","Q2","Q3","Q4"]
            usersByDevice_list = file['data']['usersByDevice']
            if usersByDevice_list is not None:
                for userBydevice in range (len(usersByDevice_list)):
                    aggre_user_state_wise['registered_user'].append(file['data']['aggregated']['registeredUsers'])
                    aggre_user_state_wise['app_open'].append(file['data']['aggregated']['appOpens'])
                    aggre_user_state_wise['year'].append(year_list[year])
                    aggre_user_state_wise['quarter'].append(quarter[json_file])
                    aggre_user_state_wise['state'].append(state)
                    aggre_user_state_wise['brand'].append(usersByDevice_list[userBydevice]['brand'])
                    aggre_user_state_wise['count'].append(usersByDevice_list[userBydevice]['count'])
                    aggre_user_state_wise['percentage'].append(usersByDevice_list[userBydevice]['percentage'])
            else:
                    aggre_user_state_wise['registered_user'].append(file['data']['aggregated']['registeredUsers'])
                    aggre_user_state_wise['app_open'].append(file['data']['aggregated']['appOpens'])
                    aggre_user_state_wise['year'].append(aggre_user_list[year])
                    aggre_user_state_wise['quarter'].append(quarter[json_file])
                    aggre_user_state_wise['state'].append(state)
                    aggre_user_state_wise['brand'].append(None)
                    aggre_user_state_wise['count'].append(0)
                    aggre_user_state_wise['percentage'].append(None)
    df_aggre_trans_year_wise = pd.DataFrame(aggre_trans_year_wise)
    df_aggre_trans_state_wise = pd.DataFrame(aggre_trans_state_wise)
    df_aggre_user_year_wise = pd.DataFrame(aggre_user_year_wise)
    df_aggre_user_state_wise = pd.DataFrame(aggre_user_state_wise)
    df1=[df_aggre_trans_year_wise,df_aggre_trans_state_wise,df_aggre_user_year_wise,df_aggre_user_state_wise]
    return df1

In [106]:
def complete_map_datas():

    #map
    #1.map transaction
    map_transaction_path = "D:/Data Science/Git_Clone/Data/pulse/data/map/transaction/hover/country/india/"
    map_transaction_list=os.listdir(map_transaction_path)

    #1.1 year wise map transaction
    map_transaction_year_wise = dict(categories=[],year=[],quarter=[],count=[],amount=[])
    for year in range(len(map_transaction_list)-1):
        json_path = map_transaction_path+map_transaction_list[year]+"/"
        json_file_list = os.listdir(json_path)
        for json_file in range(len(json_file_list)):
            df = open(f"{json_path+json_file_list[json_file]}")
            file = json.load(df)
            hoverDataList = file['data']['hoverDataList']
            quarter = ["Q1","Q2","Q3","Q4"]
            for values in hoverDataList:
                map_transaction_year_wise['categories'].append(values['name'])
                map_transaction_year_wise['year'].append(map_transaction_list[year])
                map_transaction_year_wise['quarter'].append(quarter[json_file])
                map_transaction_year_wise['count'].append(values['metric'][0]['count'])
                map_transaction_year_wise['amount'].append(values['metric'][0]['amount'])
            
    #1.2 state wise map transaction
    map_transaction_state_wise = dict(district=[],year=[],quarter=[],state=[],count=[],amount=[])
    state_path = map_transaction_path+"state"+"/"
    state_list = os.listdir(state_path)
    for state in state_list:
        year_path = state_path+state+"/"
        year_list = os.listdir(year_path)
        for year in year_list:
            json_path = year_path+year+"/"
            json_file_list = os.listdir(json_path)
            for json_file in range(len(json_file_list)):
                df = open(f"{json_path+json_file_list[json_file]}")
                file = json.load(df)
            hoverDataList = file['data']['hoverDataList']
            quarter = ["Q1","Q2","Q3","Q4"]
            for values in hoverDataList:
                map_transaction_state_wise['district'].append(values['name'])
                map_transaction_state_wise['year'].append(year)
                map_transaction_state_wise['quarter'].append(quarter[json_file])
                map_transaction_state_wise['state'].append(state)
                map_transaction_state_wise['count'].append(values['metric'][0]['count'])
                map_transaction_state_wise['amount'].append(values['metric'][0]['amount'])
              
    #2.map user
    map_user_path = "D:/Data Science/Git_Clone/Data/pulse/data/map/user/hover/country/india/"
    map_user_list=os.listdir(map_user_path)

    #2.1 year wise map user
    map_user_year_wise = dict(registered_users=[],app_open=[],state=[],year=[],quarter=[])
    for year in range(len(map_transaction_list)-1):
        json_path = map_user_path+map_user_list[year]+"/"
        json_file_list = os.listdir(json_path)
        for json_file in range(len(json_file_list)):
            df = open(f"{json_path+json_file_list[json_file]}")
            file = json.load(df)
            quarter = ["Q1","Q2","Q3","Q4"]
            state_list = ['puducherry','tamil nadu','uttar pradesh','madhya pradesh','andhra pradesh','tripura',
                     'lakshadweep','manipur','maharashtra','dadra & nagar haveli & daman & diu','meghalaya',
                     'andaman & nicobar islands','haryana','rajasthan','ladakh','punjab','assam','jharkhand',
                     'odisha','bihar','kerala','karnataka','chandigarh','telangana','himachal pradesh',
                     'west bengal','gujarat','sikkim','nagaland','mizoram','chhattisgarh','jammu & kashmir','goa',
                     'arunachal pradesh','delhi','uttarakhand']
            for state in state_list:
                map_user_year_wise['registered_users'].append(file['data']['hoverData'][state]['registeredUsers'])
                map_user_year_wise['app_open'].append(file['data']['hoverData'][state]['appOpens'])
                map_user_year_wise['year'].append(map_user_list[year])
                map_user_year_wise['state'].append(state)
                map_user_year_wise['quarter'].append(quarter[json_file])

    #2.2 state wise map user
    map_user_state_wise = dict(registered_users=[],app_open=[],state=[],district=[],year=[],quarter=[])
    state_path = map_user_path+"state"+"/"
    state_list = os.listdir(state_path)
    for state in state_list:
        year_path = state_path+state+"/"
        year_list = os.listdir(year_path)
        for year in year_list:
            json_path = year_path+year+"/"
            json_file_list = os.listdir(json_path)
            for json_file in range(len(json_file_list)):
                df = open(f"{json_path+json_file_list[json_file]}")
                file=json.load(df)
                data=list(file['data']['hoverData'].keys())
                quarter = ["Q1","Q2","Q3","Q4"]
                for values in data:
                    map_user_state_wise['state'].append(state)
                    map_user_state_wise['district'].append(values)
                    map_user_state_wise['year'].append(year)
                    map_user_state_wise['quarter'].append(quarter[json_file])
                    map_user_state_wise['registered_users'].append(file['data']['hoverData'][values]['registeredUsers'])
                    map_user_state_wise['app_open'].append(file['data']['hoverData'][values]['appOpens'])
    df_map_transaction_year_wise=pd.DataFrame(map_transaction_year_wise)
    df_map_transaction_state_wise=pd.DataFrame(map_transaction_state_wise)
    df_map_user_year_wise=pd.DataFrame(map_user_year_wise)
    df_map_user_state_wise=pd.DataFrame(map_user_state_wise)
    df2=[df_map_transaction_year_wise,df_map_transaction_state_wise,df_map_user_year_wise,df_map_user_state_wise]
    return df2
            

In [88]:
def complete_top_datas():
    #top
    #1.top transaction
    top_transaction_path = "D:/Data Science/Git_Clone/Data/pulse/data/top/transaction/country/india/"
    top_transaction_list=os.listdir(top_transaction_path)

    #1.1 year wise top transaction
    top_transaction_year_wise=dict(state=[],state_count=[],state_amount=[],year=[],quarter=[],
                                   pincode=[],pincode_amount=[],pincode_count=[],
                                   district=[],district_amount=[],district_count=[])
    for year in range(len(top_transaction_list)-1):
        json_path = top_transaction_path+top_transaction_list[year]+"/"
        json_file_list =os.listdir(json_path)
        for json_file in range(len(json_file_list)):
            df = open(f"{json_path+json_file_list[json_file]}")
            file=json.load(df)
            values1 = file['data']['states']
            values2 = file['data']['districts']
            values3 = file['data']['pincodes']
            quarter = ["Q1","Q2","Q3","Q4"]
            if len(values1) == len(values2) == len(values3) :
                for values in range(len(values1)):
                    top_transaction_year_wise['state'].append(values1[values]['entityName'])
                    top_transaction_year_wise['state_count'].append(values1[values]['metric']['count'])
                    top_transaction_year_wise['state_amount'].append(values1[values]['metric']['amount'])
                    top_transaction_year_wise['year'].append(top_transaction_list[year])
                    top_transaction_year_wise['quarter'].append(quarter[json_file])
                    top_transaction_year_wise['district'].append(values2[values]['entityName'])
                    top_transaction_year_wise['district_count'].append(values2[values]['metric']['count'])
                    top_transaction_year_wise['district_amount'].append(values2[values]['metric']['amount'])
                    top_transaction_year_wise['pincode'].append(values3[values]['entityName'])
                    top_transaction_year_wise['pincode_count'].append(values3[values]['metric']['count'])
                    top_transaction_year_wise['pincode_amount'].append(values3[values]['metric']['amount'])
                    
    #1.2 state wise top transaction
    state_path=top_transaction_path+"state"+"/"
    state_list=os.listdir(state_path) 
    top_transaction_state_wise=dict(state=[],district=[],district_count=[],district_amount=[],
                                    year=[],quarter=[],pincode=[],pincode_count=[],pincode_amount=[])
    for state in state_list:
        year_path=state_path+state+"/"
        year_list=os.listdir(year_path)
        for year in year_list:
            json_path=year_path+year+"/"
            json_file_list=os.listdir(json_path)
            for json_file in range(len(json_file_list)):
                df=open(f"{json_path+json_file_list[json_file]}")
                file=json.load(df)
                districts=file['data']['districts']
                quarter = ["Q1","Q2","Q3","Q4"]
                values1 = file['data']['districts']
                values2 = file['data']['pincodes']
                for values in range(len(values2)):
                    try:
                        top_transaction_state_wise['district'].append(values1[values]['entityName'])
                        top_transaction_state_wise['pincode'].append(values2[values]['entityName'])
                        top_transaction_state_wise['state'].append(state)
                        top_transaction_state_wise['pincode_count'].append(values2[values]['metric']['count'])
                        top_transaction_state_wise['pincode_amount'].append(values2[values]['metric']['count'])
                        top_transaction_state_wise['district_count'].append(values1[values]['metric']['count'])
                        top_transaction_state_wise['district_amount'].append(values1[values]['metric']['amount'])
                        top_transaction_state_wise['year'].append(year)
                        top_transaction_state_wise['quarter'].append(quarter[json_file])
                        
                    except IndexError:
                        top_transaction_state_wise['district'].append(None)
                        top_transaction_state_wise['pincode'].append(values2[values]['entityName'])
                        top_transaction_state_wise['state'].append(state)
                        top_transaction_state_wise['pincode_count'].append(values2[values]['metric']['count'])
                        top_transaction_state_wise['pincode_amount'].append(values2[values]['metric']['count'])
                        top_transaction_state_wise['district_count'].append(0)
                        top_transaction_state_wise['district_amount'].append(0)
                        top_transaction_state_wise['year'].append(year)
                        top_transaction_state_wise['quarter'].append(quarter[json_file])
    #2.top user
    top_user_path = "D:/Data Science/Git_Clone/Data/pulse/data/top/user/country/india/"
    top_user_list = os.listdir(top_user_path)

    #2.1 year wise top user
    top_user_path = "D:/Data Science/Git_Clone/Data/pulse/data/top/user/country/india/"
    top_user_list = os.listdir(top_user_path)
    top_user_year_wise=dict(state=[],state_registered_users=[],year=[],quarter=[],
                            district=[],district_registered_users=[],
                            pincode=[],pincode_registered_users=[])
    for year in range(len(top_user_list)-1):
        json_path = top_user_path+top_user_list[year]+"/"
        json_file_list =os.listdir(json_path)
        for json_file in range(len(json_file_list)):
            df=open(f"{json_path+json_file_list[json_file]}")
            file=json.load(df)
            states=file['data']['states']
            quarter = ["Q1","Q2","Q3","Q4"]
            values1 = file['data']['states']
            values2 = file['data']['districts']
            values3 = file['data']['pincodes']
            for values in range(len(values1)):
                top_user_year_wise['state'].append(values1[values]['name'])
                top_user_year_wise['state_registered_users'].append(values1[values]['registeredUsers'])
                top_user_year_wise['year'].append(top_user_list[year])
                top_user_year_wise['quarter'].append(quarter[json_file])
                top_user_year_wise['district'].append(values2[values]['name'])
                top_user_year_wise['district_registered_users'].append(values2[values]['registeredUsers'])
                top_user_year_wise['pincode'].append(values3[values]['name'])
                top_user_year_wise['pincode_registered_users'].append(values3[values]['registeredUsers'])
                    
    #2.2 state wise top user  
    state_path = top_user_path+"state"+"/"
    state_list = os.listdir(state_path)
    top_user_state_wise = dict(state=[],district=[],district_registered_users=[],pincode=[],pincode_registered_users=[],year=[],quarter=[])
    for state in state_list:
        year_path = state_path+state+"/"
        year_list = os.listdir(year_path)
        for year in year_list:
            json_path = year_path+year+"/"
            json_file_list = os.listdir(json_path)
            for json_file in range(len(json_file_list)):
                df = open(f"{json_path+json_file_list[json_file]}")
                file = json.load(df)
                #pprint.pprint(file)
                ditrict = file['data']['districts']
                quarter = ["Q1","Q2","Q3","Q4"]
                values1 = file['data']['districts']
                values2 = file['data']['pincodes']
                for values in range(len(values2)):
                    try:
                        top_user_state_wise['district'].append(values1[values]['name'])
                        top_user_state_wise['pincode'].append(values2[values]['name'])
                        top_user_state_wise['state'].append(state)
                        top_user_state_wise['district_registered_users'].append(values1[values]['registeredUsers'])
                        top_user_state_wise['pincode_registered_users'].append(values2[values]['registeredUsers'])
                        top_user_state_wise['year'].append(year)
                        top_user_state_wise['quarter'].append(quarter[json_file])
                    except IndexError:
                        top_user_state_wise['state'].append(state)
                        top_user_state_wise['district'].append(None)
                        top_user_state_wise['pincode'].append(values2[values]['name'])
                        top_user_state_wise['district_registered_users'].append(0)
                        top_user_state_wise['pincode_registered_users'].append(values2[values]['registeredUsers'])
                        top_user_state_wise['year'].append(year)
                        top_user_state_wise['quarter'].append(quarter[json_file])
 
    df_top_transaction_year_wise=pd.DataFrame(top_transaction_year_wise)
    df_top_transaction_state_wise=pd.DataFrame(top_transaction_state_wise)
    df_top_user_year_wise=pd.DataFrame(top_user_year_wise)
    df_top_user_state_wise=pd.DataFrame(top_user_state_wise)
    df3=[df_top_transaction_year_wise,df_top_transaction_state_wise,df_top_user_year_wise,df_top_user_state_wise]
    return df3
    

In [119]:
def create_sql_table_aggregated():
    query="create table if not exists aggre_trans_year_wise(categories varchar (250),year int,quarter varchar (10),count BIGINT,amount float)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists aggre_trans_state_wise(categories varchar (250),year int,quarter varchar (10),state varchar (50),count BIGINT,amount float)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists aggre_user_year_wise(registered_user BIGINT,app_open BIGINT,year int,quarter varchar (10),brand varchar (50),count BIGINT,percentage varchar (50)) "
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists aggre_user_state_wise(registered_user BIGINT,app_open BIGINT,year int,quarter varchar (10),state varchar (50),brand varchar (50),count BIGINT,percentage varchar (50)) "
    cursor.execute(query)
    conn.commit()
     

In [120]:
def create_sql_table_map():
    query="create table if not exists map_trans_year_wise(categories varchar (250),year int,quarter varchar (10),count BIGINT,amount float)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists map_trans_state_wise(district varchar (50),year int,quarter varchar (10),state varchar (50),count BIGINT,amount float)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists map_user_year_wise(registered_user BIGINT,app_open BIGINT,state varchar(50),year int,quarter varchar (10))"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists map_user_state_wise(registered_user BIGINT,app_open BIGINT,state varchar (50),district varchar (50),year int,quarter varchar (10))"
    cursor.execute(query)
    conn.commit()

In [136]:
def create_sql_table_top():
    query="create table if not exists top_trans_year_wise(state varchar (50),state_count bigint,state_amount float,year int,quarter varchar(10),pincode int,pincode_amount float,pincode_count bigint,district varchar(50),district_amount float,district_count bigint)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists top_trans_state_wise(state varchar (50),district varchar (50),district_count BIGINT,district_amount float,year int,quarter varchar (10),pincode int,pincode_count bigint,pincode_amount float)"
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists top_user_year_wise(state varchar(50),state_registered_user BIGINT,year int,quarter varchar (10),district varchar (50),district_registered_users BIGINT,pincode int,pincode_registered_users bigint) "
    cursor.execute(query)
    conn.commit()
    
    query="create table if not exists top_user_state_wise(state varchar (50),district varchar (50),district_registered_user BIGINT,pincode int,pincode_registered_users bigint,year int,quarter varchar (10))"
    cursor.execute(query)
    conn.commit()

In [152]:
def migrate_sql_table_aggregated():
    values=df1[0].values.tolist()
    query="insert into aggre_trans_year_wise values (%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df1[1].values.tolist()
    query="insert into aggre_trans_state_wise values (%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df1[2].values.tolist()
    query="insert into aggre_user_year_wise values (%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df1[3].values.tolist()
    query="insert into aggre_user_state_wise values (%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
  

In [153]:
def migrate_sql_table_map():
    values=df2[0].values.tolist()
    query="insert into map_trans_year_wise values (%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df2[1].values.tolist()
    query="insert into map_trans_state_wise values (%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df2[2].values.tolist()
    query="insert into map_user_year_wise values (%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df2[3].values.tolist()
    query="insert into map_user_state_wise values (%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()

In [154]:
def migrate_sql_table_top():
    values=df3[0].values.tolist()
    query="insert into top_trans_year_wise values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df3[1].values.tolist()
    query="insert into top_trans_state_wise values (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df3[2].values.tolist()
    query="insert into top_user_year_wise values (%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()
    
    values=df3[3].values.tolist()
    query="insert into top_user_state_wise values (%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,values)
    conn.commit()